In [1]:
#imports
import pandas as pd
import tensorflow
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from keras.models import Sequential, load_model
from keras import layers
from keras.metrics import AUC
from keras.callbacks import EarlyStopping
from keras.optimizer_v2.adam import Adam
import matplotlib.pyplot as plt
from pair import Pair

In [10]:
#Creating classifier labels from the closing price
def createLabels(returns: list[float], divisor: int = 1) -> list[int]:
    scalar = np.std(returns) / divisor
    labels = [int(ret / scalar) for ret in returns]
    return [l - min(labels) for l in labels]

In [11]:
# feature creation
eurusd = Pair('EURUSD', key='q0tsF6PmO8kro7SlVt0S', mode='hourly')
eurusd.ts = pd.DataFrame(eurusd.ts)

shorter = 5
longer = 15

sma_long = [None] * longer
sma_short = [None] * shorter
std_long = [None] * longer
std_short = [None] * shorter

for i in range(len(eurusd.ts) - 5):
    sma_short.append(np.mean(eurusd.ts.close[i:i+5]))
    std_short.append(np.std(eurusd.ts.close[i:i+5]))

    if not i >= len(eurusd.ts) - 15:
        sma_long.append(np.mean(eurusd.ts.close[i:i+15]))
        std_long.append(np.std(eurusd.ts.close[i:i+15]))

eurusd.ts['SMA_short'] = sma_short
eurusd.ts['STD_short'] = std_short
eurusd.ts['SMA_long'] = sma_long
eurusd.ts['STD_long'] = std_long

df = eurusd.ts[['open', 'high', 'low', 'close', 'SMA_short', 'STD_short', 'SMA_long', 'STD_long']].iloc[15:,:].reset_index(drop=True)
features = df.iloc[:-1,:]
features

,open,high,low,close,SMA_short,STD_short,SMA_long,STD_long
0,1.09516,1.09524,1.09430,1.09447,1.094062,0.000647,1.092379,0.001416
1,1.09446,1.09723,1.09445,1.09655,1.094332,0.000449,1.092641,0.001415
2,1.09655,1.09768,1.09618,1.09660,1.094850,0.000943,1.093031,0.001618
3,1.09660,1.09688,1.09425,1.09490,1.095376,0.001035,1.093396,0.001758
4,1.09491,1.09630,1.09460,1.09600,1.095534,0.000878,1.093655,0.001671
...,...,...,...,...,...,...,...,...
483,1.10386,1.10450,1.10210,1.10446,1.104492,0.000405,1.106343,0.001416
484,1.10446,1.10536,1.10425,1.10493,1.104534,0.000388,1.106221,0.001492
485,1.10492,1.10548,1.10412,1.10425,1.104616,0.000419,1.106099,0.001518
486,1.10424,1.10424,1.10356,1.10395,1.104476,0.000400,1.105891,0.001543


In [16]:
# label creation
labels_no_arr = createLabels(eurusd.logrs, 2)
label_arr = tensorflow.keras.utils.to_categorical(labels_no_arr, dtype='int64')
labels = label_arr[15:]
labels

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [17]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=.15)

ct = ColumnTransformer([('only numeric', MinMaxScaler(), features.columns)], remainder='passthrough')
features_train = ct.fit_transform(features_train)
features_test = ct.fit_transform(features_test)
all_features = ct.fit_transform(df)


In [20]:
model: Sequential
loss = 'categorical_crossentropy'
metrics = ['accuracy']
build = 1
activ = 'relu'

def build_model(name: str = 'FXc', metrics = metrics, loss = loss):
    model = Sequential(name = name)
    model.add(layers.InputLayer(input_shape=features.shape[1],))
    model.add(layers.Dense(512, activation=activ))
    # model.add(layers.Dropout(.1))
    model.add(layers.Dense(256, activation=activ))
    # model.add(layers.Dropout(.1))
    model.add(layers.Dense(128, activation=activ))
    # model.add(layers.Dropout(.1))
    model.add(layers.Dense(128, activation=activ))
    # model.add(layers.Dropout(.1))
    model.add(layers.Dense(32, activation=activ))
    # model.add(layers.Dropout(.1))
    model.add(layers.Dense(8, activation=activ))
    # model.add(layers.Dropout(.1))
    model.add(layers.Dense(label_arr.shape[1], name='Output', activation='softmax'))
    model.compile(loss=loss, optimizer=Adam(learning_rate=0.0075), metrics=metrics)
    print(f'Generating new model {model.name}')
    model.summary()
    return model

if 'model' not in dir() or build: model = build_model()
else: print('Loaded previous model:'); model.summary()

Generating new model FXc
Model: "FXc"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 512)               4608      
                                                                 
 dense_7 (Dense)             (None, 256)               131328    
                                                                 
 dense_8 (Dense)             (None, 128)               32896     
                                                                 
 dense_9 (Dense)             (None, 128)               16512     
                                                                 
 dense_10 (Dense)            (None, 32)                4128      
                                                                 
 dense_11 (Dense)            (None, 8)                 264       
                                                                 
 Output (Dense)              (None, 22

In [21]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=50, verbose=1)
history = model.fit(features_train, labels_train, batch_size=3, epochs=300, verbose=1, validation_split=.15, callbacks=[es])

Epoch 1/300
117/117 [==============================] - 3s 10ms/step - loss: 2.2517 - accuracy: 0.4587 - val_loss: 1.6946 - val_accuracy: 0.5397
Epoch 2/300
117/117 [==============================] - 1s 7ms/step - loss: 1.6195 - accuracy: 0.5442 - val_loss: 1.7185 - val_accuracy: 0.5397
Epoch 3/300
117/117 [==============================] - 0s 4ms/step - loss: 1.6068 - accuracy: 0.5442 - val_loss: 1.7052 - val_accuracy: 0.5397
Epoch 4/300
117/117 [==============================] - 1s 5ms/step - loss: 1.5846 - accuracy: 0.5442 - val_loss: 1.7643 - val_accuracy: 0.5397
Epoch 5/300
117/117 [==============================] - 1s 5ms/step - loss: 1.6276 - accuracy: 0.5442 - val_loss: 1.6972 - val_accuracy: 0.5397
Epoch 6/300
117/117 [==============================] - 1s 4ms/step - loss: 1.6038 - accuracy: 0.5442 - val_loss: 1.8804 - val_accuracy: 0.5397
Epoch 7/300
117/117 [==============================] - 0s 4ms/step - loss: 1.5881 - accuracy: 0.5442 - val_loss: 1.8244 - val_accuracy: 0.539

In [22]:
# graphs

In [23]:
y_est = model.predict(features_test)
y_est = np.argmax(y_est, axis=1)
y_true = np.argmax(labels_test, axis=1)

In [24]:
print(classification_report(y_true, y_est))

              precision    recall  f1-score   support

           9       0.00      0.00      0.00         3
          10       0.00      0.00      0.00         6
          11       0.00      0.00      0.00        10
          12       0.49      1.00      0.65        36
          13       0.00      0.00      0.00        11
          14       0.00      0.00      0.00         4
          15       0.00      0.00      0.00         3
          21       0.00      0.00      0.00         1

    accuracy                           0.49        74
   macro avg       0.06      0.12      0.08        74
weighted avg       0.24      0.49      0.32        74



/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

In [25]:
#actual is horizontal
#predicted is vertical

print(confusion_matrix(y_true, y_est))

[[ 0  0  0  3  0  0  0  0]
 [ 0  0  0  6  0  0  0  0]
 [ 0  0  0 10  0  0  0  0]
 [ 0  0  0 36  0  0  0  0]
 [ 0  0  0 11  0  0  0  0]
 [ 0  0  0  4  0  0  0  0]
 [ 0  0  0  3  0  0  0  0]
 [ 0  0  0  1  0  0  0  0]]


In [ ]:
# bayesian adjustment